02 例題アプリケーション：映画レビューのセンチメント分析
==============================================

* 本章では、Webサイトの映画レビューデータセットを用いる

    * このデータセットには、映画レビューのテキストと、そのレビューが肯定的`pos`か否定的`neg`かを示すラベルで構成されている
    
    * 1から10点の採点が行われている
    
    * モデリングを簡単にするために、この点が6点以上の場合に「肯定的」、それ以外の場合を「否定的」として、2クラス分類データセットとしている

* データを解凍すると、このデータセットが2つのフォルダに格納されたテキストファイルとして提供されていることがわかる

    * それぞれのフォルダに、さらに`pos`と`neg`というサブフォルダがある

In [3]:
!tree -dL 2 ~/data/aclImdb

/Users/MacUser/data/aclImdb
├── test
│   ├── neg
│   └── pos
└── train
    ├── neg
    ├── pos
    └── unsup

7 directories


* `pos`フォルダには肯定的なレビューが、それぞれ独立したテキストファイルとして格納されている

* `neg`フォルダには否定的なレビューが、同様に格納されている

* `unsup`フォルダにはラベルの付いていないデータが含まれるが、これは使用しないため削除する

In [4]:
!rm -r ~/data/aclImdb/train/unsup

* scikit-learnにはこのようなラベルがサブフォルダに対応するフォルダ構成からデータを読み込むための関数`load_files`が定義されている

    * まず、`load_files`関数を用いて訓練データを読み込む

In [7]:
from sklearn.datasets import load_files

reviews_train = load_files("/Users/MacUser/data/aclImdb/train/")
# load_filesは一連の訓練テキストと訓練ラベルを返す
text_train, y_train = reviews_train.data, reviews_train.target
print("type of text_train: {}".format(type(text_train)))
print("length of text_train: {}".format(len(text_train)))
print("text_train[6]:\n{}".format(text_train[6]))

type of text_train: <class 'list'>
length of text_train: 25000
text_train[6]:
b"This movie has a special way of telling the story, at first i found it rather odd as it jumped through time and I had no idea whats happening.<br /><br />Anyway the story line was although simple, but still very real and touching. You met someone the first time, you fell in love completely, but broke up at last and promoted a deadly agony. Who hasn't go through this? but we will never forget this kind of pain in our life. <br /><br />I would say i am rather touched as two actor has shown great performance in showing the love between the characters. I just wish that the story could be a happy ending."


* `text_train`はながさ25,000のリストで、個々の要素にはレビューが文字列として含まれている

    * ここではインデックス1のレビューを表示している
    
    * レビューにはHTMLの改行シーケンス(<br />)が含まれている場合がある
    
    * 取り除いておく

In [8]:
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]

* `text_train`の個々の要素の型はPythonのバージョンに依存する(Python3：文字列データのバイナリ表現、bytes型)

* このデータセットは肯定的なクラスと否定的なクラスがバランスするように集められているので、肯定的、否定的の両方に多数の文字列がある

In [10]:
import numpy as np
print("Samples per class (training): {}".format(np.bincount(y_train)))

Samples per class (training): [12500 12500]


* 同様に、テストデータセットもロードする

In [11]:
reviews_test = load_files("/Users/MacUser/data/aclImdb/test/")
text_test, y_test = reviews_test.data, reviews_test.target
print("Number of documents in test data: {}".format(len(text_test)))
print("Samples per class (test): {}".format(np.bincount(y_test)))
text_test = [doc.replace(b"<br />", b" ") for doc in text_test]

Number of documents in test data: 25000
Samples per class (test): [12500 12500]


* ここで解こうとしている問題は次の通り

    * あるレビューに対して、レビューのテキストに基づいて「肯定的」もしくは「否定的」のラベルを割り当てる
    
    * これは、標準的な2クラス分類タスクである
    
    * しかし、テキストデータは機械学習が扱えるような形式になっていない
    
    * テキストの文字列表現を機械学習のアルゴリズムを適用できるような数値表現に変換する必要がある

| 版   | 年/月/日   |
| ---- | ---------- |
| 初版 | 2019/04/06 |